## LangChain Expression Language (LCEL)


In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import BaseOutputParser

# chat = ChatOpenAI(temperature=0.1)
chat = ChatOpenAI(
    temperature = 0.1,
    streaming = True,
    callbacks = [StreamingStdOutCallbackHandler()]
)
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a world-class international chef. you create easy to follow recipies for any type of cuising with easy to find ingredient."),
    ("human", "i want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a vegetarian chef specialized on making traditional recipies vegetarian. you find alternative ingredients and explain their preparation. you don't radically modify the recipe. if there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe":chef_chain} | veg_chain
final_chain.invoke({
    "cuisine": "indian"
})
# veg_chain.invoke({
#     "recipt": "chatmodel"
# })


Great! Indian cuisine is known for its rich flavors and aromatic spices. Let's start with a classic Indian dish called Butter Chicken. Here's an easy-to-follow recipe for you:

Ingredients:
- 500g boneless chicken, cut into bite-sized pieces
- 2 tablespoons butter
- 1 onion, finely chopped
- 2 cloves of garlic, minced
- 1-inch piece of ginger, grated
- 2 teaspoons garam masala
- 1 teaspoon turmeric powder
- 1 teaspoon chili powder (adjust to your spice preference)
- 1 cup tomato puree
- 1/2 cup heavy cream
- Salt to taste
- Fresh cilantro leaves for garnish

Instructions:
1. Heat the butter in a large pan over medium heat. Add the chopped onions and sauté until they turn golden brown.
2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.
3. In a small bowl, mix together the garam masala, turmeric powder, and chili powder. Add this spice mixture to the pan and cook for a minute to release the flavors.
4. Add the chicken pieces to the pan and cook

AIMessageChunk(content="Great! Butter Chicken is a delicious dish, and I can help you make it vegetarian. Instead of using chicken, we can substitute it with a plant-based protein like tofu or paneer (Indian cottage cheese). Here's how you can modify the recipe:\n\nIngredients:\n- 500g tofu or paneer, cut into bite-sized pieces\n- 2 tablespoons butter (or vegan butter for a vegan version)\n- 1 onion, finely chopped\n- 2 cloves of garlic, minced\n- 1-inch piece of ginger, grated\n- 2 teaspoons garam masala\n- 1 teaspoon turmeric powder\n- 1 teaspoon chili powder (adjust to your spice preference)\n- 1 cup tomato puree\n- 1/2 cup coconut cream (or vegan cream)\n- Salt to taste\n- Fresh cilantro leaves for garnish\n\nInstructions:\n1. Heat the butter in a large pan over medium heat. Add the chopped onions and sauté until they turn golden brown.\n2. Add the minced garlic and grated ginger to the pan. Cook for another minute until fragrant.\n3. In a small bowl, mix together the garam masala,

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate,FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate


t= PromptTemplate(
    template = "what is the capital of {country}",
    input_variables = {"country"}
)
t.format(country = "france")

t= PromptTemplate.from_template("what is the captital of {country}")
t.format()

#유효성 검사 country!

## fewshot  
모델에게 예제를 주어준다.

In [10]:
examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

example_prompt = PromptTemplate.from_template("Human:{qustion}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt =example_prompt,
    examples=examples,
    suffix = "Human:What do you know about {country}?",
    input_variables = ["country"]
)

chain = promt | chat

chain.invoke(
    {"country": "Germany"}
)

AI:
I know this:
Capital: Berlin
Language: German
Food: Bratwurst and Sauerkraut
Currency: Euro

AIMessageChunk(content='AI:\nI know this:\nCapital: Berlin\nLanguage: German\nFood: Bratwurst and Sauerkraut\nCurrency: Euro')

## FewShotChatMessagePromptTemplate

In [13]:
examples = [
{
"country": "France",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"country": "Italy",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"country": "Greece",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "what do you know about {country}?"),
    ("ai","{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt =example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ('system', " you are a geography expert, you give short answer"),
    example_prompt,
    ("human", "what do you know about {country}?")
])
chain = final_prompt | chat

chain.invoke(
    {"country": "Germany"}
)

I know this:
Capital: Berlin
Language: German
Food: Bratwurst and Sauerkraut
Currency: Euro
Germany is known for its rich history, beautiful landscapes, and technological advancements. It is also famous for its beer culture and Oktoberfest, a traditional festival held in Munich. Germany has made significant contributions to various fields, including science, engineering, philosophy, and music. It is home to renowned composers such as Ludwig van Beethoven and Johann Sebastian Bach. Additionally, Germany is known for its efficient public transportation system and well-maintained autobahns (highways).

AIMessageChunk(content='I know this:\nCapital: Berlin\nLanguage: German\nFood: Bratwurst and Sauerkraut\nCurrency: Euro\nGermany is known for its rich history, beautiful landscapes, and technological advancements. It is also famous for its beer culture and Oktoberfest, a traditional festival held in Munich. Germany has made significant contributions to various fields, including science, engineering, philosophy, and music. It is home to renowned composers such as Ludwig van Beethoven and Johann Sebastian Bach. Additionally, Germany is known for its efficient public transportation system and well-maintained autobahns (highways).')

## LengthBasedExampleSelector

In [21]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

class RandomExampleSelector(BaseExampleSelector):
    
    def __init__(self,examples):
        self.examples = examples
        
    def add_example(self,example):
        self.example.append(example)
        
    def select_examples(self, input_variables):
        from random import choice
        return [choice(examples)]

example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

# example_selector = LengthBasedExampleSelector(
#     examples = examples,
#     example_prompt = example_prompt,
#     max_length= 80,
    
# )
example_selector = RandomExampleSelector(
    examples = examples,
    
    
)
prompt = FewShotPromptTemplate(
    example_prompt =example_prompt,
    example_selector=example_selector,
    suffix = "Human:What do you know about {country}?",
    input_variables = ["country"]
)

prompt.format(country = "brazil")
# chain = promt | chat

# chain.invoke(
#     {"country": "Germany"}
# )

'Human:What do you know about Italy?\nAI:\nI know this:\nCapital: Rome\nLanguage: Italian\nFood: Pizza and Pasta\nCurrency: Euro\n\n\nHuman:What do you know about brazil?'

## serialization and composition

In [26]:
from langchain.prompts import load_prompt

# prompt =  load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")
prompt.format(country = "korea")

'what is the capital of korea'

## pipline 


In [32]:
from langchain.prompts.pipeline import PipelinePromptTemplate

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
    """
    
)
example = PromptTemplate.from_template(
    """
    this is an example of how you talk:
    
    Human: {example_question}
    you:{example_answer}
    """
    
)
start = PromptTemplate.from_template(
    """
    Start now!
    
    Human:{question}
    You:
    """
    
)
final = PromptTemplate.from_template(
    """
    {intro}
    
    {example}
    
    {start}
    """
)
prompts = [
    ("intro",intro),
    ("example", example),
    ("start", start)
]
full_prompt= PipelinePromptTemplate(final_prompt=final,pipeline_prompts=prompts)

full_prompt.format(character="pirate",example_question="what is your location",example_answer="Arrrrrg! that is a secret. Arrrrg!",question="what is your fav food?")

chain = full_prompt | chat
chain.invoke({
    "character":"pirate","example_question":"what is your location","example_answer":"Arrrrrg! that is a secret. Arrrrg!","question":"what is your fav food?"
})

Arrrrg! Me favorite food be a good ol' plate o' fish and chips! The taste o' the salty sea be in every bite, matey! Arrrrg!

AIMessageChunk(content="Arrrrg! Me favorite food be a good ol' plate o' fish and chips! The taste o' the salty sea be in every bite, matey! Arrrrg!")